In [1]:
import sys
from IPython.display import FileLink, display
import requests
import json
import math
from fpdf import FPDF


In [2]:
%env PYTHONIOENCODING=utf-8


env: PYTHONIOENCODING=utf-8


In [ ]:
API_KEY = 'KEY'


In [4]:
GOOGLE_API_KEY = API_KEY  # Replace with your actual API key
RELEVANT_TOURIST_KEYWORDS = [
    "fort", "palace", "museum", "temple", "sanctuary", "park", "zoo", "viewpoint", "waterfall",
    "heritage", "monument", "lake", "beach", "cave", "garden", "national park", "wildlife", "trek", "hill",
    "valley", "cliff", "dam", "fountain", "art gallery", "historical", "observatory"
]
RELEVANT_CAFE_KEYWORDS = [
    "cafe", "coffee", "tea house", "bakery", "brewery", "patisserie", "espresso", "latte", "roastery", "bistro","dhaba","bhojanalaya",
    "restaurant","dining","eatery","snack bar","food court","pub","bar","tavern","canteen","kitchen"
]
RELEVANT_HOTEL_KEYWORDS = [
    "hotel", "resort", "motel", "inn", "lodge", "guest house", "homestay", "hostel",
    "suites", "retreat", "spa resort","dharamshala","ashram","rest house","dharmshala","lodging",
    "accommodation", "stay", "boarding house", "bed and breakfast", "holiday home",
    "vacation rental", "self-catering", "camping ground", "caravan park", "yurt camp"
]

# Function to get coordinates from a place name
def get_coordinates(place_name):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": place_name, "key": GOOGLE_API_KEY}
    response = requests.get(url, params=params)
    data = response.json()
    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    return None, None

def is_relevant_tourist_place(place):
    """Check if a place is relevant as a tourist attraction based on its name and types."""
    name = place.get("name", "").lower()
    types = place.get("types", [])
    return any(keyword in name for keyword in RELEVANT_TOURIST_KEYWORDS) or "tourist_attraction" in types

# Function to check if a place is a relevant cafe or restaurant
def is_relevant_cafe(place):
    """Check if a place is relevant as a cafe or restaurant."""
    name = place.get("name", "").lower()
    types = place.get("types", [])
    return any(keyword in name for keyword in RELEVANT_CAFE_KEYWORDS) or "cafe" in types or "restaurant" in types

# Function to check if a place is a relevant hotel/lodging
def is_relevant_hotel(place):
    """Check if a place is relevant as a hotel or lodging."""
    name = place.get("name", "").lower()
    types = place.get("types", [])
    return any(keyword in name for keyword in RELEVANT_HOTEL_KEYWORDS) or "lodging" in types or "hotel" in types


# Function to fetch places
def get_google_places(lat, lon, radius=25000, place_type="tourist_attraction"):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lon}", "radius": radius, "type": place_type, "key": GOOGLE_API_KEY}
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    
    places = []
    for place in results:
        photo_url = None
        if "photos" in place:
            photo_reference = place["photos"][0]["photo_reference"]
            photo_url = get_photo_url(photo_reference)
            
         # Apply correct relevance function without removing any other functionality
        if (place_type == "tourist_attraction" and is_relevant_tourist_place(place)) or \
           (place_type == "cafe" and is_relevant_cafe(place)) or \
           (place_type == "lodging" and is_relevant_hotel(place)):
            places.append({
                "name": place.get("name"),
                "rating": float(place.get("rating", 0) or 0),
                "address": place.get("vicinity", "Unknown"),
                "types": place.get("types", []),
                "lat": place["geometry"]["location"]["lat"],
                "lon": place["geometry"]["location"]["lng"],
                "photo_url": photo_url,
                "price_level": place.get("price_level", "N/A"),

            })
          
    
    return filter_by_rating(sort_places(places, lat, lon))
    
    return places

# Function to calculate number of days between two dates
def calculate_days(start_date, end_date):
    from datetime import datetime
    start = datetime.strptime(start_date, "%d-%m-%Y")
    end = datetime.strptime(end_date, "%d-%m-%Y")
    return (end - start).days + 1  # Include the end date

# Function to fetch places based on trip type
def get_places_by_trip_type(lat, lon, trip_type):
    place_type_mapping = {
        "Adventure": ["amusement_park", "campground", "hiking_area"],
        "Relaxation": ["spa", "park", "natural_feature"],
        "History": ["museum", "historical_landmark"],
        "Wildlife": ["zoo", "aquarium", "national_park"],
        "Beaches": ["beach", "natural_feature"],
        "Urban Exploration": ["shopping_mall", "tourist_attraction", "art_gallery"]
    }
    selected_types = place_type_mapping.get(trip_type, ["tourist_attraction"])
    places = []
    for place_type in selected_types:
        url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {"location": f"{lat},{lon}", "radius": 25000, "type": place_type, "key": GOOGLE_API_KEY}
        response = requests.get(url, params=params)
        results = response.json().get("results", [])
        for place in results:
            photo_url = None
            if "photos" in place:
                photo_reference = place["photos"][0]["photo_reference"]
                photo_url = get_photo_url(photo_reference)
                price_level = place.get("price_level", "N/A")
            places.append({
                "name": place.get("name"),
                "rating": place.get("rating", "N/A"),
                "address": place.get("vicinity", "Unknown"),
                "types": place.get("types", []),
                "lat": place["geometry"]["location"]["lat"],
                "lon": place["geometry"]["location"]["lng"],
                "photo_url": photo_url,
                "price_level": price_level,
                "pricing_info": get_pricing_info(price_level)
            })
    return places

# Function to estimate travel costs
def estimate_travel_cost(mode, distance):
    cost_per_km = {"car": 10, "bus": 5, "train": 8}  # Example cost per km
    return cost_per_km.get(mode, 0) * distance

# Function to calculate accommodation cost
def calculate_accommodation_cost(hotels, num_nights):
    avg_price_per_night = 3000  # Example price per night
    return avg_price_per_night * num_nights

# Function to calculate food expenses
def calculate_food_expenses(num_days, num_meals_per_day=3):
    avg_meal_cost = 300  # Example cost per meal
    return num_days * num_meals_per_day * avg_meal_cost

# Function to get photo URL
def get_photo_url(photo_reference, max_width=400):
    return f"https://maps.googleapis.com/maps/api/place/photo?maxwidth={max_width}&photo_reference={photo_reference}&key={GOOGLE_API_KEY}"

# Function to map price level to description
def get_pricing_info(price_level):
    price_mapping = {
        0: "Free",
        1: "Budget ($)",
        2: "Moderate ($$)",
        3: "Expensive ($$$)",
        4: "Luxury ($$$$)"
    }
    return price_mapping.get(price_level, "N/A")

# Function to print budget details
def print_budget_details(travel_cost, accommodation_cost, food_expenses):
    print("\n💰 **Estimated Budget:**")
    print(f"- Travel Cost: ₹{travel_cost}")
    print(f"- Accommodation Cost: ₹{accommodation_cost}")
    print(f"- Food Expenses: ₹{food_expenses}")
    print(f"**Total Estimated Cost: ₹{travel_cost + accommodation_cost + food_expenses}**")

# Functions to fetch hotels, cafes, activities, and transport facilities
def get_google_hotels(lat, lon, radius=25000):
    return get_google_places(lat, lon, radius, place_type="lodging")

def get_google_cafes(lat, lon, radius=25000):
    return get_google_places(lat, lon, radius, place_type="cafe")

def get_google_activities(lat, lon, radius=25000):
    return get_google_places(lat, lon, radius, place_type="amusement_park")

def get_google_transport(lat, lon, radius=25000):
    return get_google_places(lat, lon, radius, place_type="taxi_stand")

# Function to calculate straight-line distance using Haversine formula
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

# Helper function: Compute the average center of a list of places
def compute_center(places):
    if not places:
        return None, None
    avg_lat = sum(p['lat'] for p in places) / len(places)
    avg_lon = sum(p['lon'] for p in places) / len(places)
    return avg_lat, avg_lon

# Helper function: Optimize route using nearest neighbor heuristic
def optimize_route(places, start_lat, start_lon):
    if not places:
        return []
    unvisited = places.copy()
    route = []
    current_lat, current_lon = start_lat, start_lon
    while unvisited:
        next_place = min(unvisited, key=lambda p: calculate_distance(current_lat, current_lon, p['lat'], p['lon']))
        route.append(next_place)
        current_lat, current_lon = next_place['lat'], next_place['lon']
        unvisited.remove(next_place)
    return route

MIN_RATING = 3.8  # Set a minimum acceptable rating

def filter_by_rating(places):
    """Remove places with low ratings."""
    return [place for place in places if place.get("rating", 0) >= MIN_RATING]

def sort_places(places, center_lat, center_lon):
    return sorted(places, key=lambda p: (-float(p.get("rating", 0) or 0), calculate_distance(center_lat, center_lon, p["lat"], p["lon"])))

def generate_eating_suggestions(day_places, cafes):
    if not day_places or not cafes:
        return []
    day_center_lat = sum(p['lat'] for p in day_places) / len(day_places)
    day_center_lon = sum(p['lon'] for p in day_places) / len(day_places)
    sorted_cafes = sorted(cafes, key=lambda cafe: calculate_distance(day_center_lat, day_center_lon, cafe['lat'], cafe['lon']))
    return sorted_cafes[:2]

# Updated itinerary function:
# Now takes only one argument "all_places" and internally computes the center,
# then optimizes the route and groups the places into days (4 stops per day).
def generate_itinerary(all_places, cafes = None):
    if not all_places:
        return {}
    # Compute center of all places
    center_lat, center_lon = compute_center(all_places)
    # Optimize route starting from the computed center
    route = optimize_route(all_places, center_lat, center_lon)
    itinerary = {}
    day = 1
    # Group 4 consecutive stops per day
    for i in range(0, len(route), 4):
        day_places = route[i:i+4]
        # For each place, calculate its distance from the computed center
        itinerary[f"Day {day}"] = {
            "places": [{
                "name": p['name'],
                "rating": p['rating'],
                "distance_from_center": f"{calculate_distance(center_lat, center_lon, p['lat'], p['lon']):.2f} km"
            } for p in day_places]
        }
        day += 1

        # Group 4 consecutive stops per day
    for i in range(0, len(route), 4):
        day_places = route[i:i+4]
        day_center_lat = sum(p['lat'] for p in day_places) / len(day_places) if day_places else center_lat
        day_center_lon = sum(p['lon'] for p in day_places) / len(day_places) if day_places else center_lon
        day_eating = generate_eating_suggestions(day_places, cafes) if cafes else []
        itinerary[f"Day {day}"] = {
            "places": [{
                "name": p['name'],
                "rating": p['rating'],
                "distance_from_center": f"{calculate_distance(center_lat, center_lon, p['lat'], p['lon']):.2f} km"
            } for p in day_places],
            "eating_places": [{
                "name": cafe['name'],
                "rating": cafe['rating'],
                "distance_from_day_center": f"{calculate_distance(day_center_lat, day_center_lon, cafe['lat'], cafe['lon']):.2f} km"
            } for cafe in day_eating]
        }
        day += 1
    return itinerary


# Function to get transport distances using Distance Matrix API
def get_transport_distances(start_lat, start_lon, end_lat, end_lon):
    transport_modes = ["driving", "walking", "bicycling", "transit"]
    results = {}
    for mode in transport_modes:
        url = "https://maps.googleapis.com/maps/api/distancematrix/json"
        params = {
            "origins": f"{start_lat},{start_lon}",
            "destinations": f"{end_lat},{end_lon}",
            "mode": mode,
            "key": GOOGLE_API_KEY
        }
        response = requests.get(url, params=params)
        data = response.json()
        if data["status"] == "OK" and data["rows"][0]["elements"][0]["status"] == "OK":
            distance = data["rows"][0]["elements"][0]["distance"]["text"]
            duration = data["rows"][0]["elements"][0]["duration"]["text"]
            results[mode] = (distance, duration)
        else:
            results[mode] = ("N/A", "N/A")
    return results

# Function to display places (prints to console)
def display_places(title, places):
    print(f"\n{title}:")
    if places:
        for place in places[:15]:
            print(f"- {place['name']} ({place['rating']} ⭐) - {place.get('address', 'Unknown')} (Distance from center: {place.get('distance_from_center', 'N/A')})")
            if place.get("photo_url"):
                print(f"  Photo: {place['photo_url']}")
    else:
        print("No results found.")
        

# Function to suggest all recommended places
def suggest_all_places(places):
    grouped_places = {"Tourist Places": [], "Cafes": [], "Hotels": []}
    
    for place in places:
        if "lodging" in place["types"]:
            grouped_places["Hotels"].append(place)
        elif "cafe" in place["types"]:
            grouped_places["Cafes"].append(place)
        else:
            grouped_places["Tourist Places"].append(place)
    
    for category, places in grouped_places.items():
        print(f"\n **{category}:**")
        for place in places:
            print(f"- {place['name']} ({place['rating']}⭐) ")

# Function to print the trip details
def print_trip_details(start_location, final_destination, start_date, end_date, straight_line_distance, transport_distances):
    print(f"\n📍 **Trip Details:**")
    print(f"- Start Location: {start_location}")
    print(f"- Final Destination: {final_destination}")
    print(f"- Start Date: {start_date}")
    print(f"- End Date: {end_date}")
    print(f"\n**Distance from {start_location} to {final_destination}:**")
    print(f"- Straight-line distance: {straight_line_distance:.2f} km")
    for mode, (dist, time) in transport_distances.items():
        print(f"- {mode.capitalize()} distance: {dist} (Approx. {time})")

In [11]:
# Main Execution
start_location = input("Enter start location: ")
final_destination = input("Enter the final destination: ")

start_date = input("Enter the start date (DD-MM-YYYY): ")
end_date = input("Enter the end date (DD-MM-YYYY): ")

# Calculate number of days between start and end date
trip_duration = calculate_days(start_date, end_date) 
print(f"\n📅 Your trip duration: {trip_duration} days.")

trip_types = ["Adventure", "Relaxation", "History", "Wildlife", "Beaches", "Urban Exploration"]
trip_type = ""
while trip_type not in trip_types:
    trip_type = input("\nEnter the type of trip you want (Adventure, Relaxation, History, Wildlife, Beaches, Urban Exploration): ").strip().title()
    if trip_type not in trip_types:
        print("Invalid choice. Please select from the given options.")

start_lat, start_lon = get_coordinates(start_location)
final_lat, final_lon = get_coordinates(final_destination)
all_places = get_google_places(final_lat, final_lon)

if start_lat and start_lon and final_lat and final_lon:
    # Fetch all attractions at final destination
    all_places = get_google_places(final_lat, final_lon)

    # Filter only tourist places
    tourist_places = [place for place in all_places if is_relevant_tourist_place(place)]

    # Remove low-rated places
    filtered_tourist_places = filter_by_rating(tourist_places)

    # Sort by rating & distance
    sorted_tourist_places = sort_places(filtered_tourist_places, final_lat, final_lon)
    

    filtered_places = filter_by_rating(all_places)  # Remove low-rated places
    sorted_places = sort_places(filtered_places, final_lat, final_lon)  # Sort by rating & distance
    
    # Fetch cafes and hotels separately, ensuring at least 3-4 of each are included
    final_cafes = get_google_places(final_lat, final_lon, place_type="cafe")
    final_hotels = get_google_hotels(final_lat, final_lon)
    
 
    combined_places = all_places + final_cafes[:5] + final_hotels[:5]

    # Fetch places based on trip type
    trip_places = get_places_by_trip_type(final_lat, final_lon, trip_type)

    # Ensure itinerary does not exceed trip duration
    itinerary = generate_itinerary(combined_places)  # Now, only all_places is passed
    limited_itinerary = {day: itinerary[day] for day in list(itinerary.keys())[:trip_duration]}

    #Distances between two places
    straight_line_distance = calculate_distance(start_lat, start_lon, final_lat, final_lon)
    transport_distances = get_transport_distances(start_lat, start_lon, final_lat, final_lon)
    
    # Filter only tourist places
    tourist_places = [place for place in all_places if is_relevant_tourist_place(place)]

    # Ensure itinerary does not exceed trip duration
    tour_itinerary = generate_itinerary(tourist_places)  # Now, only tourist places are passed
    limited_tour_itinerary = {day: tour_itinerary[day] for day in list(tour_itinerary.keys())[:trip_duration]}

    print_trip_details(start_location, final_destination, start_date, end_date, straight_line_distance, transport_distances)
    
    suggest_all_places(combined_places)
    print("\n🌟 **Recommended Itinerary (Tourist Places with Eating Options):**")
    for day, details in limited_itinerary.items():
        print(f"### {day}")
        print("Tourist Attractions:")
        for activity in details['places']:
            print(f"- {activity['name']} ({activity['rating']}*) - Distance from center: {activity['distance_from_center']}")
        if details.get("eating_places"):
            print("Eating Places:")
            for eat in details['eating_places']:
                print(f"- {eat['name']} ({eat['rating']}*) - Distance from day center: {eat['distance_from_day_center']}")

def generate_itinerary_pdf(itinerary, trip_details):
            # ✅ Correctly Register ArialSans (with Full Unicode Support)  
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(0, 10, "Trip Itinerary", ln=True, align="C")
    pdf.ln(5)
    pdf.set_font("Arial", size=12)
    pdf.cell(0, 10, f"Start: {trip_details['start_location']}", ln=True)
    pdf.cell(0, 10, f"Destination: {trip_details['final_destination']}", ln=True)
    pdf.cell(0, 10, f"Dates: {trip_details['start_date']} to {trip_details['end_date']}", ln=True)
    pdf.cell(0, 10, f"Straight-line distance: {trip_details['straight_line_distance']:.2f} km", ln=True)
    pdf.ln(10)
    for day, details in itinerary.items():
        pdf.set_font("Arial", "B", 14)
        pdf.cell(0, 10, day, ln=True)
        pdf.set_font("Arial", size=12)
        pdf.cell(0, 8, "Tourist Attractions:", ln=True)
        for activity in details['places']:
            pdf.multi_cell(0, 8, f"- {activity['name']} ({activity['rating']}*) - {activity['distance_from_center']}")
        if details.get("eating_places"):
            pdf.cell(0, 8, "Eating Places:", ln=True)
            for eat in details['eating_places']:
                pdf.multi_cell(0, 8, f"- {eat['name']} ({eat['rating']}*) - {eat['distance_from_day_center']}")
        pdf.ln(5)
    filename = "itinerary.pdf"
    pdf.output(filename)
    return filename

trip_details = {
    "start_location": start_location,
    "final_destination": final_destination,
    "start_date": start_date,
    "end_date": end_date,
    "straight_line_distance": straight_line_distance
}

# Estimate travel cost
chosen_transport = "car"  # Replace with user input if needed
travel_distance = float(transport_distances.get(chosen_transport, (0, ""))[0])  # Extract numeric distance
travel_cost = estimate_travel_cost(chosen_transport, travel_distance)

# Calculate accommodation cost
num_nights = trip_duration
accommodation_cost = calculate_accommodation_cost(final_hotels, num_nights)

# Calculate food expenses
food_expenses = calculate_food_expenses(trip_duration)

# Print budget details
print_budget_details(travel_cost, accommodation_cost, food_expenses)

# Generate itinerary PDF
trip_details = {
    "start_location": start_location,
    "final_destination": final_destination,
    "start_date": start_date,
    "end_date": end_date,
    "straight_line_distance": straight_line_distance
}
itinerary_pdf = generate_itinerary_pdf(limited_itinerary, trip_details)

# Writing output to a file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write("Trip Itinerary\n")
    f.write(f"Start: {trip_details['start_location']}\n")
    f.write(f"Destination: {trip_details['final_destination']}\n")
    f.write(f"Dates: {trip_details['start_date']} to {trip_details['end_date']}\n")
    f.write(f"Straight-line distance: {trip_details['straight_line_distance']:.2f} km\n\n")

    for day, details in limited_itinerary.items():
        f.write(f"### {day}\n")
        f.write("Tourist Attractions:\n")
        for activity in details['places']:
            f.write(f"- {activity['name']} ({activity['rating']}*) - Distance from center: {activity['distance_from_center']}\n")
        if details.get("eating_places"):
            f.write("Eating Places:\n")
            for eat in details['eating_places']:
                f.write(f"- {eat['name']} ({eat['rating']}*) - Distance from day center: {eat['distance_from_day_center']}\n")
        f.write("\n")
    
    with open("output.txt", "w", encoding="utf-8") as f:
        f.write("")
    pdf_link = FileLink("output.txt")
    display(pdf_link)

print("Itinerary PDF and text file generated successfully.")



📅 Your trip duration: 7 days.

📍 **Trip Details:**
- Start Location: delhi
- Final Destination: goa
- Start Date: 20-02-2025
- End Date: 26-02-2025

**Distance from delhi to goa:**
- Straight-line distance: 1514.09 km
- Driving distance: 1,908 km (Approx. 1 day 8 hours)
- Walking distance: 1,744 km (Approx. 16 days 9 hours)
- Bicycling distance: N/A (Approx. N/A)
- Transit distance: 1,600 km (Approx. 21 hours 16 mins)

 **Tourist Places:**
- Shri Mahalasa Narayani Devalaya (4.8⭐) 
- Shri Shantadurga Kunkallikarin Temple (4.8⭐) 
- Shri Mahalasa Narayani Temple (4.8⭐) 
- Shree Saunsthan Shantadurga Chamundeshwari Kudtari Mahamaya (4.7⭐) 
- Shri Chandreshwar Bhootnath Temple (4.7⭐) 
- Shri Ramnath Devasthan, Bandora (4.7⭐) 
- Shri Nageshi Temple, (4.7⭐) 
- Our Lady of Health Church (4.6⭐) 
- Dudhsagar Falls (4.6⭐) 
- Holy Trinity Church (4.6⭐) 
- Our Lady of Mount Carmel Chapel (4.5⭐) 
- Saviour of the World Church (4.4⭐) 
- Big Foot Goa (4.1⭐) 
- Bondla Wildlife Sanctuary (4.0⭐) 
- Frog

d:\SoftwareEngineering\Project\output.txt

Itinerary PDF and text file generated successfully.
